In [1]:
# Complex Models Using the Keras Functional API
# For this notebooks we used the California housing problem and tackle it using three ways.
# 1) Wide & Deep Neural Network
# 2) Wide & Deep Neural Network with 2 inputs
# 3) Wide & Deep Neural Network with 2 inputs and 2 outputs
# The focus is on the illustration of the Functional API from Keras, not on achieving any high metric

In [2]:
# The Wide & Deep Neural Network is able to connect all or part of the inputs directly to the output layer,
# hence the Keras Sequential API cannot be used for it.

In [3]:
# Import our Set up
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

import tensorflow as tf
from tensorflow import keras

In [4]:
housing = fetch_california_housing()
X_train_full, X_test, y_train_full, y_test = train_test_split(housing.data, housing.target)
X_train, X_valid, y_train, y_valid = train_test_split(X_train_full, y_train_full)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_valid = scaler.transform(X_valid)
X_test = scaler.transform(X_test)

In [11]:
# Model1 - Wide & Deep Neural Network
input_ = keras.layers.Input(shape=X_train.shape[1:])
hidden1 = keras.layers.Dense(30, activation="relu")(input_)
hidden2 = keras.layers.Dense(30, activation="relu")(hidden1)
concat = keras.layers.Concatenate()([input_, hidden2])
output = keras.layers.Dense(1)(concat)
model1 = keras.Model(inputs=[input_], outputs=[output])

# Compiling and Evaluating the Model
optimizer = keras.optimizers.SGD(learning_rate=0.00001, nesterov=False, name='SGD')
model1.compile(loss="mean_squared_error", optimizer=optimizer)
history = model1.fit(X_train, y_train, epochs=100, validation_data=(X_valid, y_valid))
mse_test = model1.evaluate(X_test, y_test)

# Pretend these are new instances
X_new = X_test[:3]
y_pred = model1.predict(X_new)

Epoch 1/100
363/363 [==============================] - 1s 1ms/step - loss: 5.3080 - val_loss: 5.2671
Epoch 2/100
363/363 [==============================] - 0s 1ms/step - loss: 5.0911 - val_loss: 5.0443
Epoch 3/100
363/363 [==============================] - 0s 1ms/step - loss: 4.8843 - val_loss: 4.8326
Epoch 4/100
363/363 [==============================] - 0s 1ms/step - loss: 4.6867 - val_loss: 4.6318
Epoch 5/100
363/363 [==============================] - 0s 1ms/step - loss: 4.4981 - val_loss: 4.4406
Epoch 6/100
363/363 [==============================] - 0s 1ms/step - loss: 4.3175 - val_loss: 4.2588
Epoch 7/100
363/363 [==============================] - 0s 1ms/step - loss: 4.1449 - val_loss: 4.0858
Epoch 8/100
363/363 [==============================] - 0s 1ms/step - loss: 3.9798 - val_loss: 3.9212
Epoch 9/100
363/363 [==============================] - 0s 1ms/step - loss: 3.8217 - val_loss: 3.7645
Epoch 10/100
363/363 [==============================] - 0s 1ms/step - loss: 3.6704 - val_lo

In [12]:
print(y_pred)

[[2.3133554]
 [2.103262 ]
 [1.9170855]]


In [13]:
# Model2 - Wide & Deep Neural Network with 2 inputs
input_A = keras.layers.Input(shape=[5], name="wide_input")
input_B = keras.layers.Input(shape=[6], name="deep_input")
hidden1 = keras.layers.Dense(30, activation="relu")(input_B)
hidden2 = keras.layers.Dense(30, activation="relu")(hidden1)
concat = keras.layers.concatenate([input_A, hidden2])
output = keras.layers.Dense(1, name="output")(concat)
model2 = keras.Model(inputs=[input_A, input_B], outputs=[output])

In [15]:
# We have to pass a dictionary mapping the input names to the input values
X_train_A, X_train_B = X_train[:, :5], X_train[:, 2:]
X_valid_A, X_valid_B = X_valid[:, :5], X_valid[:, 2:]
X_test_A, X_test_B = X_test[:, :5], X_test[:, 2:]
X_new_A, X_new_B = X_test_A[:3], X_test_B[:3]

train_dict = {"wide_input": X_train_A, "deep_input": X_train_B}
valid_dict = {"wide_input": X_valid_A, "deep_input": X_valid_B}
test_dict = {"wide_input": X_test_A, "deep_input": X_test_B}
new_dict = {"wide_input": X_new_A, "deep_input": X_new_B}

model2.compile(loss="mse", optimizer=optimizer)
history = model2.fit(train_dict, y_train, epochs=20, validation_data=(valid_dict, y_valid))
mse_test = model2.evaluate(test_dict, y_test)
y_pred = model2.predict(new_dict)

Epoch 1/20
363/363 [==============================] - 1s 1ms/step - loss: 3.6393 - val_loss: 3.5964
Epoch 2/20
363/363 [==============================] - 0s 1ms/step - loss: 3.5496 - val_loss: 3.5126
Epoch 3/20
363/363 [==============================] - 0s 1ms/step - loss: 3.4623 - val_loss: 3.4312
Epoch 4/20
363/363 [==============================] - 0s 1ms/step - loss: 3.3773 - val_loss: 3.3523
Epoch 5/20
363/363 [==============================] - 0s 1ms/step - loss: 3.2946 - val_loss: 3.2757
Epoch 6/20
363/363 [==============================] - 0s 1ms/step - loss: 3.2140 - val_loss: 3.2013
Epoch 7/20
363/363 [==============================] - 0s 1ms/step - loss: 3.1355 - val_loss: 3.1292
Epoch 8/20
363/363 [==============================] - 0s 1ms/step - loss: 3.0591 - val_loss: 3.0592
Epoch 9/20
363/363 [==============================] - 0s 1ms/step - loss: 2.9847 - val_loss: 2.9912
Epoch 10/20
363/363 [==============================] - 0s 1ms/step - loss: 2.9122 - val_loss: 2.9252

In [16]:
print(y_pred)

[[0.90867376]
 [0.64235437]
 [1.0258842 ]]


In [17]:
# Model2 - Wide & Deep Neural Network with 2 inputs and 2 outputs
input_A = keras.layers.Input(shape=[5], name="wide_input")
input_B = keras.layers.Input(shape=[6], name="deep_input")
hidden1 = keras.layers.Dense(30, activation="relu")(input_B)
hidden2 = keras.layers.Dense(30, activation="relu")(hidden1)
concat = keras.layers.concatenate([input_A, hidden2])
output = keras.layers.Dense(1, name="main_output")(concat)
aux_output = keras.layers.Dense(1, name="aux_output")(hidden2)
model3 = keras.Model(inputs=[input_A, input_B], outputs=[output, aux_output])

In [19]:
# Each output needs its own loss function. Therefore, when we compile the model, we should pass a list of losses.
# We care much more about the main output than about the auxiliary output (most of the cases it is used for regularization), 
# so we wanto to give the main output's loss a much grater weight.
# The different losses, weight factors, y_valid, y_test and y_train are in a form of a dictionary as the inputs earlier.
# We use the data preprocessing we did for the model2.

In [22]:
# Dictionaries for losses, loss_weights, y_valid, y_test and y_train
loss_dict = {"main_output": "mse", "aux_output": "mse"}
y_train_dict = {"main_output": y_train, "aux_output": y_train}
weights_dict = {"main_output": 0.9, "aux_output": 0.1}
y_valid_dict = {"main_output": y_valid, "aux_output": y_valid}
y_test_dict = {"main_output": y_test, "aux_output": y_test}

In [28]:
# Compiling and Evaluating the Model
model3.compile(loss=loss_dict, loss_weights=weights_dict, optimizer=optimizer)
history = model3.fit(train_dict, y_train_dict, epochs=20, validation_data=(valid_dict, y_valid_dict))
total_loss, main_loss, aux_loss = model3.evaluate(test_dict, y_test_dict)
y_pred_main, y_pred_aux = model3.predict(new_dict)

Epoch 1/20
363/363 [==============================] - 1s 2ms/step - loss: 3.1100 - main_output_loss: 2.9042 - aux_output_loss: 4.9622 - val_loss: 3.5456 - val_main_output_loss: 3.3834 - val_aux_output_loss: 5.0053
Epoch 2/20
363/363 [==============================] - 1s 1ms/step - loss: 3.0561 - main_output_loss: 2.8455 - aux_output_loss: 4.9521 - val_loss: 3.4973 - val_main_output_loss: 3.3306 - val_aux_output_loss: 4.9971
Epoch 3/20
363/363 [==============================] - 1s 1ms/step - loss: 3.0038 - main_output_loss: 2.7885 - aux_output_loss: 4.9417 - val_loss: 3.4498 - val_main_output_loss: 3.2789 - val_aux_output_loss: 4.9885
Epoch 4/20
363/363 [==============================] - 1s 2ms/step - loss: 2.9530 - main_output_loss: 2.7333 - aux_output_loss: 4.9305 - val_loss: 3.4033 - val_main_output_loss: 3.2282 - val_aux_output_loss: 4.9795
Epoch 5/20
363/363 [==============================] - 1s 1ms/step - loss: 2.9036 - main_output_loss: 2.6796 - aux_output_loss: 4.9191 - val_loss

In [29]:
print(y_pred_main)
print(y_pred_aux)

[[2.305287  ]
 [0.74481755]
 [1.5974648 ]]
[[1.0650762 ]
 [0.1066421 ]
 [0.15939891]]
